In [129]:
ARCHITECTURES = ["ssr3", "csr3", "ssr2", "csr2"]
BROWSERS = ["chrome", "edge"]
PAGES = ["home", "category", "recipe"]
INSTANCE_SIZES = [5, 10, 15, 20, 25, 30]
PLATFORMS = ["desktop", "mobile"]
METRICS = ["FCP", "LCP", "SI", "TBT", "SRT"]
OUTPUT_FOLDER_MEAN = "out_mean"
OUTPUT_FOLDER_MEDIAN = "out_median"

In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from scipy.stats import trim_mean
import os

In [131]:
def read_json():
    result = dict()
    for architecture in ARCHITECTURES:
        if architecture not in result.keys():
            result[architecture] = dict()
        for browser in BROWSERS:
            if browser not in result[architecture].keys():
                result[architecture][browser] = dict()
            for page in PAGES:
                if browser not in result[architecture][browser].keys():
                    result[architecture][browser][page] = dict()
                for platform in PLATFORMS:
                    table = dict()
                    for instance_size in INSTANCE_SIZES:
                        with open("../lighthouse-mag/reports/" + architecture + "/" + browser + "/" + page + "/" + platform + "/" + str(instance_size) + ".json", "r") as json_file:
                            data = json.load(json_file)
                            table[instance_size] = data
                    data_frame = pd.DataFrame(table).transpose()
                    result[architecture][browser][page][platform] = data_frame
    return result

In [132]:
def read_csv():
    result = {"Architecture": [], "Browser": [], "Page": [], "Platform": [], "Instance size": [], "FCP": [], "LCP": [], "SI": [], "TBT": [], "SRT": []}
    for architecture in ARCHITECTURES:
        for browser in BROWSERS:
            for page in PAGES:
                for platform in PLATFORMS:
                    for instance_size in INSTANCE_SIZES:
                        data = np.genfromtxt("../lighthouse-mag/reports/" + architecture + "/" + browser + "/" + page + "/" + platform + "/" + str(instance_size) + ".csv", delimiter=',', dtype=None, names=True)
                        for row in data:
                            result["Architecture"].append(architecture)
                            result["Browser"].append(browser)
                            result["Page"].append(page)
                            result["Platform"].append(platform)
                            result["Instance size"].append(instance_size)
                            result["FCP"].append(row[0])
                            result["LCP"].append(row[1])
                            result["SI"].append(row[2])
                            result["TBT"].append(row[3])
                            result["SRT"].append(row[4])
    return pd.DataFrame(result)

In [133]:
json_data = read_json()
csv_data = read_csv()

In [134]:
data_grouped = csv_data.groupby(['Architecture', 'Browser', 'Page', 'Platform', 'Instance size'])
trimmed_mean = data_grouped.agg(lambda x: trim_mean(x, 0.2))
median = data_grouped.agg("median")
min = data_grouped.agg("min")
max = data_grouped.agg("max")
std = data_grouped.agg("std") 

In [135]:
def draw_chart(browser, page, platform, metric, architectures=ARCHITECTURES):
    data_mean = trimmed_mean.groupby(['Browser', 'Page', 'Platform']).get_group((browser, page, platform)).reset_index()[['Architecture', 'Instance size', metric]]
    data_median = median.groupby(['Browser', 'Page', 'Platform']).get_group((browser, page, platform)).reset_index()[['Architecture', 'Instance size', metric]]
    min_value = float(min.groupby(['Browser', 'Page', 'Platform']).get_group((browser, page, platform)).reset_index()[['Architecture', 'Instance size', metric]].agg('min')[metric])
    max_value = float(max.groupby(['Browser', 'Page', 'Platform']).get_group((browser, page, platform)).reset_index()[['Architecture', 'Instance size', metric]].agg('max')[metric])
    std_value = float(std.groupby(['Browser', 'Page', 'Platform']).get_group((browser, page, platform)).reset_index()[['Architecture', 'Instance size', metric]].agg('max')[metric])
    ###Generate mean plots
    for architecture in architectures:
        subset = data_mean[data_mean["Architecture"] == architecture]
        plt.plot(subset["Instance size"], subset[metric], marker='o', label=architecture)
    plt.xlabel("Rozmiar instancji")
    plt.ylabel(metric + " [ms]")
    plt.ylim(min_value - std_value, max_value + std_value)
    plt.title("Konfiguracja: " + browser + ", " + page + ", " + platform)
    plt.legend(title="Architektura")
    plt.grid(True)
    if (not os.path.isdir("./" + OUTPUT_FOLDER_MEAN)):
        os.makedirs("./" + OUTPUT_FOLDER_MEAN)
    plt.savefig("./" + OUTPUT_FOLDER_MEAN + "/" + browser + "-" + page + "-" + platform + "-" + metric + ".png", bbox_inches='tight')
    plt.close()
    ##Generate median plots
    for architecture in architectures:
        subset = data_median[data_median["Architecture"] == architecture]
        plt.plot(subset["Instance size"], subset[metric], marker='o', label=architecture)
    plt.xlabel("Rozmiar instancji")
    plt.ylabel(metric + " [ms]")
    plt.ylim(min_value - std_value, max_value + std_value)
    plt.title("Konfiguracja: " + browser + ", " + page + ", " + platform)
    plt.legend(title="Architektura")
    plt.grid(True)
    if (not os.path.isdir("./" + OUTPUT_FOLDER_MEDIAN)):
        os.makedirs("./" + OUTPUT_FOLDER_MEDIAN)
    plt.savefig("./" + OUTPUT_FOLDER_MEDIAN + "/" + browser + "-" + page + "-" + platform + "-" + metric + ".png", bbox_inches='tight')
    plt.close()

In [136]:
for browser in BROWSERS:
    for page in PAGES:
        for platform in PLATFORMS:
            for metric in METRICS:
                draw_chart(browser, page, platform, metric)
